In [ ]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from weight_estimation.weight_estimator import WeightEstimator, CameraMetadata
from research_lib.utils.data_access_utils import S3AccessUtils, RDSAccessUtils

In [ ]:
s3 = S3AccessUtils('/root/data')
rds = RDSAccessUtils()

In [ ]:
query = """
    SELECT *
    FROM 
        prod.biomass_computations
    WHERE
        pen_id = 145 AND
        akpd_score > 0.95 AND
        captured_at BETWEEN '2021-02-10' and '2021-02-15'
"""

df = rds.extract_from_database(query)

In [ ]:
weight_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb'
small_weight_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/small_fish_weight_model.pb'
kf_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb'

weight_model_f, _, _ = s3.download_from_url(weight_model_url)
small_weight_model_f, _, _ = s3.download_from_url(small_weight_model_url)
kf_model_f, _, _ = s3.download_from_url(kf_model_url)
weight_estimator = WeightEstimator(weight_model_f, small_weight_model_f, kf_model_f)

weights = []
count = 0
for idx, row in df.iterrows():
    annotation = row.annotation
    cm = row.camera_metadata
    camera_metadata = CameraMetadata(
        focal_length=cm['focalLength'],
        focal_length_pixel=cm['focalLengthPixel'],
        baseline_m=cm['baseline'],
        pixel_count_width=cm['pixelCountWidth'],
        pixel_count_height=cm['pixelCountHeight'],
        image_sensor_width=cm['imageSensorWidth'],
        image_sensor_height=cm['imageSensorHeight']
    )
    
    weight, length, kf = weight_estimator.predict(annotation, camera_metadata)
    weights.append(weight)
    
    if count % 100 == 0:
        print(count)
    count += 1
    
    


In [ ]:
weight_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/trained_models/2020-11-27T00-00-00/weight_model_synthetic_data.pb'
small_weight_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/biomass/playground/small_fish_weight_model.pb'
kf_model_url = 'https://aquabyte-models.s3-us-west-1.amazonaws.com/k-factor/trained_models/2020-08-08T000000/kf_predictor_v2.pb'

weight_model_f, _, _ = s3.download_from_url(weight_model_url)
small_weight_model_f, _, _ = s3.download_from_url(small_weight_model_url)
kf_model_f, _, _ = s3.download_from_url(kf_model_url)
weight_estimator = WeightEstimator(weight_model_f, small_weight_model_f, kf_model_f)

weights_2 = []
count = 0
for idx, row in df.iterrows():
    annotation = row.annotation
    cm = row.camera_metadata
    camera_metadata = CameraMetadata(
        focal_length=cm['focalLength'],
        focal_length_pixel=cm['focalLengthPixel'],
        baseline_m=cm['baseline'] * 1.007,
        pixel_count_width=cm['pixelCountWidth'],
        pixel_count_height=cm['pixelCountHeight'],
        image_sensor_width=cm['imageSensorWidth'],
        image_sensor_height=cm['imageSensorHeight']
    )
    
    weight, length, kf = weight_estimator.predict(annotation, camera_metadata)
    weights_2.append(weight)
    
    if count % 100 == 0:
        print(count)
    count += 1
    
    


In [ ]:
np.mean(weights)

In [ ]:
np.mean(weights_2)

In [ ]:
(7612.7259523413 - 7476.883013775673) / 7476.883013775673